# Cineworld Listings

I like Weird movies coming to Imax, I don't like watching newspapers/etc to see Belfast announcing them. 

## Basic mode

Yup, turns out cineworld just checks based on user-agent. And overall appears to be even more basic that [what this one looked like](https://github.com/oracal/cineworld)

Easy observations:

* Films are uniquely identified by `id`, which is also persisted in teh rest of the api calls (see `link` attribute)
* `117` is Belfasts Site code
* No clue what the 10108 is.

In [92]:
import requests
from datetime import date

headers = {
    "User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:124.0) Gecko/20100101 Firefox/124.0",
}

requests.get(f"https://www.cineworld.co.uk/uk/data-api-service/v1/quickbook/10108/film-events/in-cinema/117/at-date/{date.today().isoformat()}", 
             headers=headers).json()['body']['films'][0]

{'id': 'ho00011281',
 'name': 'Ghilli (Re-release) (Tamil)',
 'length': 164,
 'posterLink': 'https://regalcdn.azureedge.net/CW/GhilliRereleaseTamil/HO00011281/TV_SmallPosterImage/20240415-120139903.jpg',
 'videoLink': 'https://youtu.be/4aGEjyn-bPQ?si=CgXZSe1WH5Cc4292',
 'link': 'https://www.cineworld.co.uk/films/ghilli-re-release-tamil/ho00011281',
 'weight': 10,
 'releaseYear': '2024',
 'attributeIds': ['12a',
  '2d',
  'action',
  'drama',
  'reserved-selected',
  'subbed']}

In [93]:
from datetime import date
def get_cinema_listing_for(screening_date = None, site_code=117):
    if screening_date is None:
        screening_date = str(date.today())
    elif isinstance(screening_date, date):
        screening_date = str(screening_date)
    else:
        raise ValueError("Can only use date-type with screening_date")
    response = requests.get(f"https://www.cineworld.co.uk/uk/data-api-service/v1/quickbook/10108/film-events/in-cinema/{site_code}/at-date/{screening_date}", headers=headers)
    response.raise_for_status()
    return response.json()['body']['films']
    
listings = get_cinema_listing_for()
listings[0]['name']

'Ghilli (Re-release) (Tamil)'

In [96]:
listings[0].keys()

dict_keys(['id', 'name', 'length', 'posterLink', 'videoLink', 'link', 'weight', 'releaseYear', 'attributeIds', 'date'])

In [94]:
import pandas as pd
from tqdm.auto import tqdm

listings = []

for d in tqdm(pd.date_range('today', freq='D', periods=28)):
    _d = d.date()
    for listing in get_cinema_listing_for(_d):
        listing['date'] = _d
        listings.append(listing)        

  0%|          | 0/28 [00:00<?, ?it/s]

In [84]:
df = pd.DataFrame(listings)

In [85]:
df[df['attributeIds'].apply(lambda l: 'imax' in l)][['name','date']]

,name,date
2,Civil War,2024-04-15
15,Civil War,2024-04-16
27,Civil War,2024-04-17
42,Civil War,2024-04-18
54,(IMAX) Hunger Games: Ballad Of Songbirds & Snakes,2024-04-20
55,(IMAX) Oppenheimer,2024-04-20
56,(IMAX) Spider-Man: Across The Spider-Verse,2024-04-20
57,(IMAX) The Super Mario Bros. Movie,2024-04-20
67,SPY x FAMILY CODE: White (Subtitled),2024-04-27
70,SPY x FAMILY CODE: White (Subtitled),2024-04-28
